In [76]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [107]:
umsatz_csv_url = 'https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/refs/heads/main/umsatzdaten_gekuerzt.csv'
df_umsatzdaten = pd.read_csv(umsatz_csv_url)

#Einlesen von kiwo.csv
kiwo_csv_url = 'https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/refs/heads/main/kiwo.csv'
df_kiwo = pd.read_csv(kiwo_csv_url)

#Einlesen von wetter.csv
wetter_csv_url = 'https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/refs/heads/main/wetter.csv'
df_wetter = pd.read_csv(wetter_csv_url)

#Einlesen von test.csv
test_url = 'test.csv'
df_test = pd.read_csv(test_url)

# source: https://www.finanz-tools.de/inflation/inflationsraten-deutschland

In [108]:
df_umsatzdaten = df_umsatzdaten.drop("Umsatz", axis=1)
df_umsatzdaten = df_umsatzdaten.drop("Warengruppe", axis=1)

In [101]:
df_umsatzdaten.to_csv('output_csv', index=False)

In [102]:
import pandas as pd
from datetime import datetime, timedelta

def extend_dates_until_july_2019(df):
    """
    Erweitert den Datumsbereich des Dataframes bis zum 31.07.2019
    
    Args:
        df: DataFrame mit den Spalten 'id' und 'Datum'
        
    Returns:
        DataFrame mit erweitertem Datumsbereich
    """
    # Sicherstellen dass das Datum im richtigen Format ist
    df['Datum'] = pd.to_datetime(df['Datum'])
    
    # Zieldatum erstellen
    target_date = pd.to_datetime('2019-07-31')
    
    # Alle existierenden Datums-IDs extrahieren
    existing_ids = set(df['id'].astype(str))
    
    # Neue Zeilen vorbereiten
    new_rows = []
    
    # Startdatum (ein Tag nach dem letzten vorhandenen Datum)
    current_date = df['Datum'].max() + timedelta(days=1)
    
    while current_date <= target_date:
        # ID-Pattern für jedes Datum generieren (1-6)
        for i in range(1, 7):
            # ID im Format YYMMDD{i} erstellen
            year = current_date.strftime('%y')
            month = current_date.strftime('%m')
            day = current_date.strftime('%d')
            new_id = f"{year}{month}{day}{i}"
            
            # Nur hinzufügen wenn die ID noch nicht existiert
            if new_id not in existing_ids:
                new_rows.append({
                    'id': new_id,
                    'Datum': current_date
                })
                existing_ids.add(new_id)
        
        # Zum nächsten Tag gehen
        current_date += timedelta(days=1)
    
    # Neue Zeilen als DataFrame
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        # Ursprünglichen DataFrame mit neuem kombinieren
        result_df = pd.concat([df, new_df], ignore_index=True)
        # Nach Datum sortieren
        result_df = result_df.sort_values(['Datum', 'id'])
        return result_df
    
    return df

extended_df = extend_dates_until_july_2019(df_umsatzdaten)

In [104]:
extended_df.to_csv('extended_csv', index=False)

In [54]:
extended_df

,id,Datum
0,1307011,2013-07-01
1,1307021,2013-07-02
2,1307031,2013-07-03
3,1307041,2013-07-04
4,1307051,2013-07-05
...,...,...
11519,1907312,2019-07-31
11520,1907313,2019-07-31
11521,1907314,2019-07-31
11522,1907315,2019-07-31


In [55]:
import pandas as pd

def encode_weekdays(df):
    """
    Erstellt für jeden Wochentag eine neue Spalte mit 1 oder 0,
    basierend auf dem Datum in der 'Datum'-Spalte.
    
    Parameters:
    df (pandas.DataFrame): DataFrame mit einer 'Datum'-Spalte im Format 'YYYY-MM-DD'
    
    Returns:
    pandas.DataFrame: Original DataFrame mit zusätzlichen Wochentags-Spalten
    """
    # Stelle sicher, dass die Datum-Spalte als datetime-Typ vorliegt
    df['Datum'] = pd.to_datetime(df['Datum'])
    
    # Erstelle für jeden Wochentag eine neue Spalte
    weekdays = {
        'Monday': 0,
        'Tuesday': 1,
        'Wednesday': 2,
        'Thursday': 3,
        'Friday': 4,
        'Saturday': 5,
        'Sunday': 6
    }
    
    for day, day_num in weekdays.items():
        df[day] = (df['Datum'].dt.dayofweek == day_num).astype(int)
    
    return df

In [56]:
df_umsatz_codiert = encode_weekdays(extended_df)

In [47]:
df_umsatz_codiert

,id,Datum,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1307011,2013-07-01,1,0,0,0,0,0,0
1,1307021,2013-07-02,0,1,0,0,0,0,0
2,1307031,2013-07-03,0,0,1,0,0,0,0
3,1307041,2013-07-04,0,0,0,1,0,0,0
4,1307051,2013-07-05,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
11519,1907312,2019-07-31,0,0,1,0,0,0,0
11520,1907313,2019-07-31,0,0,1,0,0,0,0
11521,1907314,2019-07-31,0,0,1,0,0,0,0
11522,1907315,2019-07-31,0,0,1,0,0,0,0


In [22]:
df_umsatzdaten

,id,Datum,Warengruppe,Umsatz,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1307011,2013-07-01,1,148.828353,1,0,0,0,0,0,0
1,1307021,2013-07-02,1,159.793757,0,1,0,0,0,0,0
2,1307031,2013-07-03,1,111.885594,0,0,1,0,0,0,0
3,1307041,2013-07-04,1,168.864941,0,0,0,1,0,0,0
4,1307051,2013-07-05,1,171.280754,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9329,1712216,2017-12-21,6,87.471228,0,0,0,1,0,0,0
9330,1712226,2017-12-22,6,71.911652,0,0,0,0,1,0,0
9331,1712236,2017-12-23,6,84.062223,0,0,0,0,0,1,0
9332,1712246,2017-12-24,6,60.981969,0,0,0,0,0,0,1
